In [ ]:
import numpy as np
import random
import gurobipy as gp
import time

Generate Random Instances using capacity mean and standard deviation

In [ ]:
def generate_instances(n, p, capacity_mean, capacity_stddev):
    # Generate 2D positions for n nodes
    nodes = np.random.rand(n, 2)

    # Generate capacities for clusters
    capacities = np.random.normal(capacity_mean, capacity_stddev, p)

    return nodes, capacities

Generate Random Weights using weight mean and standard deviation

In [ ]:
def generate_weights(n, weight_mean, weight_stddev):
    # Generate weights for n nodes
    weights = np.random.normal(weight_mean, weight_stddev, n)

    return weights

 Euclidean distance between two points p and q

In [ ]:
def distance(p, q):
    return np.sqrt(np.sum((p-q)**2))

In [ ]:
def solve_ccp(nodes, capacities, weights, lambda_param):
    n = nodes.shape[0]
    p = len(capacities)

    # Create gurobi model
    model = gp.Model('ccp')

    # Create decision variables
    x = {}
    y = {}

    # Update decision variables as mentioned in the problem statement
    for i in range(n):
        for j in range(p):
            x[i, j] = model.addVar(vtype=gp.GRB.BINARY, name=f'x[{i},{j}]')
        y[i] = model.addVar(vtype=gp.GRB.BINARY, name=f'y[{i}]')

    # Set objective function
    obj = gp.quicksum(distance(nodes[i], nodes[j]) * x[i,j] for i in range(n) for j in range(p))
    obj += lambda_param * gp.quicksum(capacities[j]*y[j] for j in range(p))
    obj -= lambda_param * gp.quicksum(weights[i]*x[i,j] for i in range(n) for j in range(p))
    model.setObjective(obj, gp.GRB.MINIMIZE)

    # Add constraints
    for i in range(n):
        model.addConstr(gp.quicksum(x[i,j] for j in range(p)) == 1, name=f'assign[{i}]')

    model.addConstr(gp.quicksum(y[j] for j in range(p)) <= p, name='num_clusters')

    for i in range(n):
        for j in range(p):
            model.addConstr(x[i,j] <= y[j], name=f'x_c[{i},{j}]')

    # Solve model
    model.optimize()
    # Extract solution
    clusters = []
    for j in range(p):
        cluster = [i for i in range(n) if x[i,j].X > 0.5]
        clusters.append(cluster)

    return clusters, model

In [ ]:
# Example 
np.random.seed(42)

# Number of nodes
n = 200
# Number of clusters
p = 2
# Metrics
capacity_mean = 10
capacity_stddev = 2
weight_mean = 1
weight_stddev = 0.1
lambda_param = 0.5

# Generate 2D nodes and random capacities for clusters
nodes, capacities = generate_instances(n, p, capacity_mean, capacity_stddev)
# Generate random weight for each node
weights = generate_weights(n, weight_mean, weight_stddev)

clusters, model = solve_ccp(nodes, capacities, weights, lambda_param)
print(clusters)